![Start](image/Twitter1.JPG)
![Start2](image/Twitter2.JPG)

In [ ]:
%%bash
pip install python-twitter
apt-get update
apt-get install mplayer vim
if ! [ -f /usr/local/bin/speech.sh ]
then
cat << 'EOF' > /usr/local/bin/speech.sh
#!/bin/bash                                                                                                                        
say() { local IFS=+;/usr/bin/mplayer -ao alsa -really-quiet -noconsolecontrols "http://translate.google.com/translate_tts?ie=UTF-8&client=tw-ob&q=$*&tl=es"; }
say $*
EOF
chmod +x /usr/local/bin/speech.sh
fi
speech.sh HOLA

# RasPI Selfie to Twitter

In [ ]:
import twitter #twitter
import picamera #picamer
import grovepi #general input, output grovePI
import grove_rgb_lcd #GrovePI LCD
from time import sleep #Sleep
import os #Import os to use for google talk
import chestnut.graph as graph #grap api


def write_lcd(text):
    try:
       grove_rgb_lcd.setRGB(255,0,0)
       grove_rgb_lcd.setText(text) 
    except IOError:
        pass
    
def close_lcd():
    try:
       grove_rgb_lcd.setRGB(0,0,0)
       grove_rgb_lcd.setText("") 
    except IOError:
        pass 
    
def google_talk(text):
    os.system("speech.sh "+text)

def take_photo():
    write_lcd("Foto!!!!")
    google_talk("Os voy a hacer una foto")
    for count in range(3,0,-1):
        out = grovepi.digitalWrite(led_pin, 1)
        write_lcd(str(count))
        google_talk(str(count))
        sleep(0.2)
        out = grovepi.digitalWrite(led_pin , 0)
        sleep(0.2)
    for count in range(4):
        out = grovepi.digitalWrite(buzzer_pin , 1)
        sleep(0.2)
        out = grovepi.digitalWrite(buzzer_pin , 0)
    grovepi.digitalWrite(led_pin, 1)
    sleep(1)
    camera.capture(imagefile)
    api.PostUpdate(status="Foto #iotpue", media=imagefile )
    out = grovepi.digitalWrite(7, 0)
    close_lcd()
    google_talk("Podéis ver la foto en mi twitter @RaspiPUEDAY")  

#Slots    
led_pin = 7
buzzer_pin = 6
button_pin = 5
ultrasonic_ranger = 4

#Variables
distance_take_foto = 10 #distance in cm to take de photo
count = 4 #Count to take de foto

images_dir = "image" #dir to put the photo
imagefile = images_dir+'/pueday.jpg' #name of the photo

#PINS INPUT_OUTPUT
grovepi.pinMode(button_pin, 'INPUT')
grovepi.pinMode(led_pin, 'OUTPUT')
grovepi.pinMode(buzzer_pin, 'OUTPUT')
grovepi.pinMode(ultrasonic_ranger, 'INPUT')

#LEDS OFF, BUZZER OFF, LCD OFF
grovepi.digitalWrite(buzzer_pin , 0)
grovepi.digitalWrite(led_pin , 0)
close_lcd()

#Open picamera
camera = picamera.PiCamera()

#Twitter api https://dev.twitter.com replace with your keys
api = twitter.Api(
    consumer_key='',
    consumer_secret='',
    access_token_key='',
    access_token_secret=''
)



chart = graph.Line(['Distancia'])

while True:
    try:
        distance = grovepi.ultrasonicRead(ultrasonic_ranger)
        chart.update([distance])
        button = grovepi.digitalRead(button_pin)
        if button == 1 or distance < distance_take_foto :
            take_photo()
        sleep(0.5)
    except KeyboardInterrupt:
        camera.close()
        out = grovepi.digitalWrite(buzzer_pin , 0)
        out = grovepi.digitalWrite(led_pin , 0)
        break
    except TypeError:
        distant = 1000
    except:
        camera.close()
        out = grovepi.digitalWrite(buzzer_pin , 0)
        out = grovepi.digitalWrite(led_pin , 0)
        print("Unexpected error:", sys.exc_info()[0])
        raise